In [26]:
import os
import PIL
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import h5py
import cv2
%matplotlib inline

In [27]:
labels = []

In [28]:
filename = None

for filename in range(1, 3065):
    with h5py.File('./brain_tumor_dataset/imageData/{}.mat'.format(filename), 'r') as f:
        label = f['cjdata']['label'][0][0] - 1
        labels.append(int(label))

In [29]:
for i in range(len(labels)):
    if(labels[i]==0):
        labels[i] = "glioma"
    elif(labels[i]==1):
        labels[i] = "meningioma"
    else:
        labels[i] = "pituitary"

In [30]:
def importing_data(path):
    sample = []
    for filename in os.listdir(path):
        if "jpg" in filename:
            sample.append(filename)
    return sample

In [31]:
path = './brain_tumor_dataset/bt_images/'
train = importing_data(path)

train_data = pd.DataFrame({'image':train, 'label': labels})

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_val = train_test_split(train_data, test_size = 0.1, shuffle = True, random_state = 42)

In [33]:
IMG_SIZE = 64
size = (IMG_SIZE,IMG_SIZE)

In [34]:
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPooling2D, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD

In [35]:
inputs = Input(shape = (64, 64, 1))
conv1 =  Conv2D(32, (3,3), activation = 'tanh', padding = 'same', strides=(1,1), input_shape = [64,64,1])(inputs)
pool1 = MaxPooling2D(2)(conv1)
conv2 = Conv2D(64, (3,3), activation = 'tanh', padding = 'same', strides=(1,1))(pool1)
pool2 = MaxPooling2D(2)(conv2)
conv3 = Conv2D(64, (3,3), activation = 'tanh', padding = 'same', strides=(1,1))(pool2)
pool3 = MaxPooling2D(2)(conv3)
conv4 = Conv2D(128,(3,3), activation = 'tanh', padding = 'same', strides=(1,1))(pool3)
pool4 = MaxPooling2D(2)(conv4)
conv5 = Conv2D(256,(3,3), activation = 'tanh', padding = 'same', strides=(1,1))(pool4)
pool5 = MaxPooling2D(2)(conv5)
flatten = Flatten()(pool5)
dense1 = Dense(32, activation = 'tanh')(flatten)
dropout1 = Dropout(0.25)(dense1)
outputs = Dense(3, activation = 'softmax')(dropout1)
model = Model(inputs=inputs, outputs=outputs)


model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0     

In [36]:
datagen = ImageDataGenerator(rescale = 1./255)

train_set = datagen.flow_from_dataframe(X_train,
                                        directory = './brain_tumor_dataset/bt_images/',
                                        x_col = 'image',
                                        y_col = 'label',
                                        target_size = size,
                                        color_mode = 'grayscale',
                                        class_mode = 'categorical',
                                        batch_size = 1,
                                        shuffle = True,
                                        interpolation = 'bilinear')

val_set = datagen.flow_from_dataframe(X_val,
                                      directory = './brain_tumor_dataset/bt_images/',
                                      x_col = 'image',
                                      y_col = 'label',
                                      target_size = size,
                                      color_mode = 'grayscale',
                                      class_mode = 'categorical',
                                      batch_size = 1,
                                      shuffle = True,
                                      interpolation = 'bilinear')

    

model.compile(optimizer="sgd",loss='categorical_crossentropy',metrics=['accuracy']) 
es = EarlyStopping(monitor='val_loss', mode='min', patience=2, restore_best_weights=True, verbose=1)
checkpoint_cb = ModelCheckpoint("./models/bayesian.h5", save_best_only=True)
history = model.fit(train_set, validation_data = val_set, epochs= 20, callbacks=[checkpoint_cb, es])   

Found 2757 validated image filenames belonging to 3 classes.
Found 307 validated image filenames belonging to 3 classes.
Epoch 1/20
2757/2757 [==============================] - 48s 17ms/step - loss: 0.7497 - accuracy: 0.6685 - val_loss: 0.5929 - val_accuracy: 0.7231
Epoch 2/20
2757/2757 [==============================] - 46s 17ms/step - loss: 0.5279 - accuracy: 0.7842 - val_loss: 0.5055 - val_accuracy: 0.7980
Epoch 3/20
2757/2757 [==============================] - 53s 19ms/step - loss: 0.4072 - accuracy: 0.8368 - val_loss: 0.4827 - val_accuracy: 0.8176
Epoch 4/20
2757/2757 [==============================] - 48s 17ms/step - loss: 0.3206 - accuracy: 0.8803 - val_loss: 0.2318 - val_accuracy: 0.9055
Epoch 5/20
2757/2757 [==============================] - 46s 17ms/step - loss: 0.2310 - accuracy: 0.9180 - val_loss: 0.3523 - val_accuracy: 0.8925
Epoch 6/20
2757/2757 [==============================] - 45s 16ms/step - loss: 0.1810 - accuracy: 0.9405 - val_loss: 0.3976 - val_accuracy: 0.8697
Res

In [43]:
def preprocess(img):
    img = img.astype(np.float32)
    img /= 255.0
    img = np.expand_dims(img,0)
    return img

In [44]:
val = X_val["image"].values.tolist()

In [45]:
class_names = {0: "glioma", 1: "meningoma", 2:"pituitary"}
d = 1
iters = 600
popsize = 10
model_path = "./models/bayesian.h5"

In [78]:
import keras 
keras_model = keras.models.load_model(model_path)
keras_model.compile(optimizer = "sgd", loss='categorical_crossentropy', metrics =['accuracy'])

In [68]:
def perturb(x):
    adv_img = orig.copy()

    pixs = np.array(np.split(x, len(x)/3)).astype(int)
    loc = (pixs[:, 0], pixs[:,1])
    val = pixs[:, 2]
    adv_img[loc] = val
    adv_img = np.expand_dims(adv_img,0)
    return adv_img

In [69]:
def optimize(x):
    adv_img = perturb(x)

    prob = keras_model.predict(preprocess(adv_img[0]))
    pred_orig = np.argmax(prob_orig)

    return prob[0][pred_orig]

In [70]:
pred_adv = 0
prob_adv = 0
ct = 0
def callback(x, convergence):
    global pred_adv, prob_adv, ct
    adv_img = perturb(x)


    prob = keras_model.predict(preprocess(adv_img[0]))

    pred_adv = np.argmax(prob)
    prob_adv = prob[0][pred_adv]
    if pred_adv != pred_orig:
        ct = ct+1
        return True

In [71]:
from scipy.optimize import differential_evolution

for filename in val:
    pred_adv = 0
    prob_adv = 0
    orig = cv2.imread(os.path.join("./brain_tumor_dataset/bt_images/",filename))
    orig = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
    orig = cv2.resize(orig, size)
    
    img = orig.copy()
    img = img.reshape(64,64,1)
    img = np.expand_dims(img,0)
    shape = img.shape
    
    
    prob_orig = keras_model.predict(preprocess(img[0]))
    pred_orig = np.argmax(prob_orig)

    bounds = [(0, shape[1]-1), (0, shape[2]-1), (0, 255)] * d
    result = differential_evolution(optimize, bounds, maxiter=iters, popsize=popsize, tol=1e-5, callback=callback)

    adv_img = perturb(result.x)


    prob = keras_model.predict(preprocess(adv_img[0]))
    print('Prob [%s]: %f --> Prob[%s]: %f' %(class_names[pred_orig], prob_orig[0][pred_orig], class_names[pred_adv], prob_adv))

    if(pred_orig != pred_adv):
        cv2.imwrite('./bayesian_outputs/'+filename, adv_img[0])

Prob [pituitary]: 0.901734 --> Prob[pituitary]: 0.806878
Prob [glioma]: 0.904938 --> Prob[glioma]: 0.796120
Prob [pituitary]: 0.982158 --> Prob[pituitary]: 0.961883
Prob [pituitary]: 0.959297 --> Prob[pituitary]: 0.942415
Prob [meningoma]: 0.665961 --> Prob[pituitary]: 0.507128
Prob [glioma]: 0.984873 --> Prob[glioma]: 0.979944
Prob [meningoma]: 0.988818 --> Prob[meningoma]: 0.985793
Prob [pituitary]: 0.787028 --> Prob[pituitary]: 0.675887
Prob [glioma]: 0.682220 --> Prob[glioma]: 0.578912
Prob [pituitary]: 0.682680 --> Prob[pituitary]: 0.615938
Prob [meningoma]: 0.985713 --> Prob[meningoma]: 0.980416
Prob [meningoma]: 0.990916 --> Prob[meningoma]: 0.989114
Prob [glioma]: 0.906895 --> Prob[glioma]: 0.849717
Prob [pituitary]: 0.902777 --> Prob[pituitary]: 0.819178
Prob [meningoma]: 0.961846 --> Prob[meningoma]: 0.952047
Prob [meningoma]: 0.991218 --> Prob[meningoma]: 0.989799
Prob [meningoma]: 0.956741 --> Prob[meningoma]: 0.949698
Prob [meningoma]: 0.987661 --> Prob[meningoma]: 0.98436

Prob [pituitary]: 0.625959 --> Prob[pituitary]: 0.518598
Prob [glioma]: 0.515899 --> Prob[meningoma]: 0.565002
Prob [meningoma]: 0.815260 --> Prob[meningoma]: 0.754106
Prob [meningoma]: 0.992194 --> Prob[meningoma]: 0.990823
Prob [meningoma]: 0.947393 --> Prob[meningoma]: 0.924781
Prob [meningoma]: 0.531600 --> Prob[pituitary]: 0.531350
Prob [pituitary]: 0.880019 --> Prob[pituitary]: 0.821537
Prob [pituitary]: 0.971088 --> Prob[pituitary]: 0.958992
Prob [meningoma]: 0.680097 --> Prob[meningoma]: 0.545762
Prob [glioma]: 0.976345 --> Prob[glioma]: 0.959182
Prob [meningoma]: 0.864588 --> Prob[meningoma]: 0.775403
Prob [glioma]: 0.597112 --> Prob[meningoma]: 0.537471
Prob [pituitary]: 0.839114 --> Prob[pituitary]: 0.788646
Prob [meningoma]: 0.968989 --> Prob[meningoma]: 0.955738
Prob [glioma]: 0.744770 --> Prob[glioma]: 0.642030
Prob [pituitary]: 0.966416 --> Prob[pituitary]: 0.945066
Prob [meningoma]: 0.716715 --> Prob[meningoma]: 0.659017
Prob [meningoma]: 0.873255 --> Prob[meningoma]: 0

Prob [meningoma]: 0.996108 --> Prob[meningoma]: 0.995642
Prob [glioma]: 0.990569 --> Prob[glioma]: 0.986458
Prob [meningoma]: 0.741138 --> Prob[meningoma]: 0.624828
Prob [glioma]: 0.992756 --> Prob[glioma]: 0.991519
Prob [pituitary]: 0.919682 --> Prob[pituitary]: 0.880199
Prob [pituitary]: 0.974575 --> Prob[pituitary]: 0.958114
Prob [glioma]: 0.941342 --> Prob[glioma]: 0.924174
Prob [glioma]: 0.798852 --> Prob[glioma]: 0.686370
Prob [pituitary]: 0.986998 --> Prob[pituitary]: 0.983021
Prob [meningoma]: 0.916465 --> Prob[meningoma]: 0.870718
Prob [meningoma]: 0.920977 --> Prob[meningoma]: 0.840763


In [72]:
print(ct)

25


In [73]:
datagen = ImageDataGenerator(rescale = 1./255)
adv_set = datagen.flow_from_dataframe(X_val,
                                      directory = './bayesian_outputs/',
                                      x_col = 'image',
                                      y_col = 'label',
                                      target_size = size,
                                      color_mode = 'grayscale',
                                      class_mode = 'categorical',
                                      batch_size = 1,
                                      shuffle = True,
                                      interpolation = 'bilinear')

Found 25 validated image filenames belonging to 3 classes.


D:\Softwares\anaconda\envs\tf-gpu\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:279: UserWarning: Found 282 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  warnings.warn(


In [74]:
predict = keras_model.evaluate(adv_set)

25/25 [==============================] - 1s 6ms/step - loss: 0.7349 - accuracy: 0.5600


In [79]:
path = './bayesian_outputs/'
adv = importing_data(path)
adv_df = pd.DataFrame({'image':adv})
val_df = X_val[~X_val.image.isin(adv_df.image)]

In [85]:
val_set = datagen.flow_from_dataframe(val_df,
                                      directory = './brain_tumor_dataset/bt_images/',
                                      x_col = 'image',
                                      y_col = 'label',
                                      target_size = size,
                                      color_mode = 'grayscale',
                                      class_mode = 'categorical',
                                      batch_size = 1,
                                      shuffle = True,
                                      interpolation = 'bilinear')

Found 282 validated image filenames belonging to 3 classes.


In [86]:
keras_model.evaluate(val_set)

282/282 [==============================] - 1s 5ms/step - loss: 0.1906 - accuracy: 0.9362


[0.1906488686800003, 0.936170220375061]

In [77]:
es = EarlyStopping(monitor='val_loss', mode='min', patience=2, restore_best_weights=True, verbose=1)

checkpoint_cb = ModelCheckpoint("./models/bayesian_adversarial.h5", save_best_only=True)

history_adv = keras_model.fit(adv_set, validation_data = val_set, epochs= 50, batch_size = 1, callbacks=[es, checkpoint_cb, reduce_lr])   

Epoch 1/50
25/25 [==============================] - 2s 65ms/step - loss: 1.2527 - accuracy: 0.4400 - val_loss: 0.5815 - val_accuracy: 0.7557
Epoch 2/50
25/25 [==============================] - 1s 57ms/step - loss: 0.3514 - accuracy: 0.8400 - val_loss: 0.2971 - val_accuracy: 0.8925
Epoch 3/50
25/25 [==============================] - 1s 57ms/step - loss: 0.0631 - accuracy: 1.0000 - val_loss: 0.3145 - val_accuracy: 0.8762
Epoch 4/50
25/25 [==============================] - 1s 58ms/step - loss: 0.0511 - accuracy: 1.0000 - val_loss: 0.3198 - val_accuracy: 0.8827
Restoring model weights from the end of the best epoch.
Epoch 00004: early stopping


In [ ]:
from scipy.optimize import differential_evolution

for filename in val:
    pred_adv = 0
    prob_adv = 0
    orig = cv2.imread(os.path.join("./brain_tumor_dataset/bt_images/",filename))
    orig = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
    orig = cv2.resize(orig, size)
    
    img = orig.copy()
    img = img.reshape(64,64,1)
    img = np.expand_dims(img,0)
    shape = img.shape
    
    
    prob_orig = keras_model.predict(preprocess(img[0]))
    pred_orig = np.argmax(prob_orig)

    bounds = [(0, shape[1]-1), (0, shape[2]-1), (0, 255)] * d
    result = differential_evolution(optimize, bounds, maxiter=iters, popsize=popsize, tol=1e-5, callback=callback)

    adv_img = perturb(result.x)

    prob = keras_model.predict(preprocess(adv_img[0]))
    print('Prob [%s]: %f --> Prob[%s]: %f ' %(class_names[pred_orig], prob_orig[0][pred_orig], class_names[pred_adv], prob_adv))

In [ ]:
calculate_ssim(adv_img[0], orig)